In [1]:
from ipymarkup import show_dep_ascii_markup, show_dep_markup, show_span_box_markup
from ipymarkup.palette import palette, BLUE, RED, GREEN
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Syntax
from yargy import Parser

In [2]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
import yake
from laserembeddings import Laser

In [ ]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
syntax = Syntax.load('slovnet_syntax_news_v1.tar')
syntax.navec(navec)

In [ ]:
#text = "За последние 10 лет, во втором десятилетии XXI века, рынок стартапов и венчурного инвестирования для многих незаметно, но кардинально изменился. Парадигмой пионеров интернета было создать проект, который изменит интернет к лучшему — «сделать новый Google» (новый Facebook, новый YouTube, новый айфон и так далее). Новой парадигмой стартапера, задумывающегося о своём вкладе в интернет, вступившей в силу в последнее десятилетие, стало создать проект, который будет куплен «Гуглом», «Фейсбуком», Apple (или «Яндексом» или Mail.ru в случае Рунета). Об изменении мира или хотя бы интернета речи уже не идёт — мир уже изменился, а в процессе был открыт, изучен и поделён. Все точки входа и торговые пути под контролем выросших за первые два десятилетия XXI века империй, и новоприбывающим колонистам ничего не остаётся, кроме как выбирать, флагу какой из них присягнуть. Для многих людей, включая автора этих строк, заставших нынешние интернет-империи «в коротких штанишках», когда мы дышали с ними одним воздухом интернет-вольницы, а «Гугл» ещё верил в свой девиз Don't be evil, произошедшее преображение оказалось неприятным, болезненным и неожиданным поворотом. И — отрезвляющим. Но, на самом деле, за последние 10 лет не произошло совершенно ничего нового."
text = "Минтранс предложил оснастить автомобили сервисов такси устройствами для контроля сонливости водителей. Об этом говорится в проекте федерального закона «О такси», размещённого на сайте проектов нормативно-правовых актов. Законопроект предусматривает, чтобы кузов такси окрашивали по схеме из квадратов контрастных цветов в шахматном порядке. При этом цветовая гамма должна «соответствовать стандартам региона». На крыше автомобиля необходимо размещать опознавательный фонарь оранжевого цвета. Если проект примут, он вступит в силу 1 сентября 2022 года. Действующие ПДД разрешают водителю находиться за рулём суммарно не более десяти часов. Контролируют это с помощью тахографов. Штраф для нарушителей — до 2,5 тысяч рублей."

In [ ]:
chunk = []
sentences = []
for sent in sentenize(text):
    sentence = {}
    sentence['text'] = sent.text
    sentence['span'] = (sent.start, sent.stop)
    sentences.append(sentence)
    tokens = [_.text for _ in tokenize(sent.text)]
    chunk.append(tokens)
chunk[:1]

In [ ]:
markup = next(syntax.map(chunk))

In [ ]:
# Convert CoNLL-style format to source, target indices
words, deps = [], []
for token in markup.tokens:
    words.append(token.text)
    source = int(token.head_id) - 1
    target = int(token.id) - 1
    if source > 0 and source != target:  # skip root, loops
        deps.append([source, target, token.rel])
show_dep_ascii_markup(words, deps)

In [ ]:
show_dep_markup(words, deps)

In [ ]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [ ]:
sentences

In [ ]:
sent_texts = []
for s in sentences:
    sent_texts.append(s['text'])

In [ ]:
sent_texts

In [ ]:
results = model.predict(sent_texts)

In [ ]:
delta = 0.001
spans = []
for sentence, sentiment in zip(sentences, results):
    print(sentence['text'], '\n', sentiment)
    main_sentiment = ''
    if sentiment['negative'] > sentiment['positive']:
        main_sentiment = 'Negative'
    if sentiment['positive'] > sentiment['negative']:
        main_sentiment = 'Positive'
    if abs(sentiment['positive'] - sentiment['negative']) < delta:
        main_sentiment = 'Neutral'
    if main_sentiment != '':
        spans.append(sentence['span'] + (main_sentiment,))

In [ ]:
spans

In [ ]:
show_span_box_markup(text, spans, palette=palette(RED, GREEN, BLUE))

In [ ]:
language = "ru"
max_ngram_size = 4
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

In [ ]:
kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)

In [ ]:
keywords = kw_extractor.extract_keywords(text)
keywords.sort(key = lambda t: t[1])
for kw in keywords :
    print(kw)

In [ ]:
limit = 0.01

for (text, value) in keywords :
    if (value < limit) :
        print(text, ' : ', value)

In [ ]:
laser = Laser()

In [ ]:
embeddings = laser.embed_sentences(sent_texts, lang='en')

In [ ]:
print(embeddings)